## Here we have provided simulation for 2-qubits CNOT and 3-qubits CCNOT gate based on EPR

In [ ]:
!pip install qiskit

In [1]:
from qiskit import QuantumCircuit
import numpy as np
from numpy import sqrt
from qiskit.visualization import plot_bloch_multivector
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector, DensityMatrix

In [2]:
# To properly observe the the operator and states
def drawState(psi):
    return psi.draw('latex', prefix='|\\psi\\rangle = ')

def drawOperator(rho):
    return rho.draw('latex', prefix='\\rho = ')

In [46]:
#Pauli operators (1-qubit) and Hadamard
X = qi.Operator.from_label('X')
Y = qi.Operator.from_label('Y')
Z = qi.Operator.from_label('Z')
I = qi.Operator.from_label('I')
H = qi.Operator.from_label('H')

In [47]:
def mul(A, B):#Matrix multiplication
  return np.matmul(A,B)

pi = np.pi

$$
R_x(\theta) = e^{-i \frac{\theta}{2} X}=\cos\left(\frac{\theta}{2}\right) I - i \sin\left(\frac{\theta}{2}\right) X
$$
$$
R_y(\theta) = e^{-i \frac{\theta}{2} Y}=\cos\left(\frac{\theta}{2}\right) I - i \sin\left(\frac{\theta}{2}\right) Y
$$
$$
R_z(\theta) = e^{-i \frac{\theta}{2} Z}=\cos\left(\frac{\theta}{2}\right) I - i \sin\left(\frac{\theta}{2}\right) Z
$$
$$
R_{zz}(\theta) = e^{-i \frac{\theta}{2} (Z \otimes Z)}=\cos\left(\frac{\theta}{2}\right) I - i \sin\left(\frac{\theta}{2}\right) (Z \otimes Z)
$$



# 1. CNOT gate

In [17]:
I4 = I.tensor(I) # 4x4 identity matrix
def exponential2(theta, A):#to calculate the unitary operator for 2 qubits (e^(iAx)=cos(x)I+i sin(x)A)
  m = np.cos(theta)
  if m<pow(10, -5):
    m=0
  a = DensityMatrix(m*I4)
  b = DensityMatrix(np.sin(theta)*A)
  return a+1j*b

In [16]:
ZI = Z.tensor(I)
IZ = I.tensor(Z)
XI = X.tensor(I)
IX = I.tensor(X)
ZZ = Z.tensor(Z)
XX = X.tensor(X)
IY = I.tensor(Y)
YI = Y.tensor(I)

####  Control-Z or U_phi for CNOT

In [10]:
B_prime = exponential2(-pi/4, I4)
Rz1z2 = exponential2(-pi/4, ZZ)
Rx2 = exponential2(0, IX)
Rx1 = exponential2(0, XI)
Rz2 = exponential2(pi/4, IZ)
Rz1 = exponential2(pi/4, ZI)

In [11]:
c0 = mul(Rz1z2, B_prime)
c1 = mul(Rx2,c0)
c2 = mul(Rx1,c1)
c3 = mul(Rz2,c2)
c4 = mul(Rz1,c3)

U_phi = DensityMatrix(c4)
drawOperator(U_phi)

<IPython.core.display.Latex object>

#### Pseudo-Hadamard (U_H')

In [12]:
U_H_prime = exponential2(-pi/4,IY)
U_H_prime_dagger = exponential2(pi/4,IY)

### Final CNOT

In [44]:
c5 = mul(U_phi,U_H_prime_dagger)
c6 = mul(U_H_prime, c5)

U_CNOT = DensityMatrix(c6)

In [45]:
drawOperator(U_CNOT)

<IPython.core.display.Latex object>

# 2. CCNOT

In [18]:
I8 = I.tensor(I4)# 8x8 identity matrix
def exponential3(theta, A):#to calculate the unitary operators for 3 qubits (e^(iAx)=cos(x)I+i sin(x)A)
  m = np.cos(theta)
  if m<pow(10, -5):
    m=0
  a = DensityMatrix(m*I8)
  b = DensityMatrix(np.sin(theta)*A)
  return a+1j*b

In [25]:
IIZ = (I.tensor(I)).tensor(Z)
IZI = (I.tensor(Z)).tensor(I)
ZII = (Z.tensor(I)).tensor(I)

ZZI = (Z.tensor(Z)).tensor(I)
IZZ = (I.tensor(Z)).tensor(Z)
ZIZ = (Z.tensor(I)).tensor(Z)

IYI = I.tensor(Y.tensor(I))
S2S3 = DensityMatrix(mul(IZI, IIZ))
S1S3 = DensityMatrix(mul(ZII, IIZ))
S1S2 = DensityMatrix(mul(ZII, IZI))
YII = Y.tensor(I.tensor(I))
IIY = I.tensor(I.tensor(Y))

##### Here there are 2 kinds of gates - a) CNOT b) Controlled- X^(1/2), now first we have to create U_phi for both CNOT and Controlled- X^(1/2)

####  Calculate U<sub>ϕ</sub> for CNOT when omega t = pi

In [26]:
v0 = exponential3(-np.pi/4, I8)
v1 = exponential3(-np.pi/4, IZZ)
v2 = exponential3(-np.pi/4, ZIZ)
v3 = exponential3(-np.pi/4, ZZI)
v4 = exponential3(np.pi/4, IIZ)
v5 = exponential3(np.pi/4, IZI)
v6 = exponential3(np.pi/4, ZII)

In [27]:
n0 = mul(v1,v0)
n1 = mul(v2,n0)
n2 = mul(v3,n1)
n3 = mul(v4,n2)
n4 = mul(v5,n3)
n5 = mul(v6,n4)


U_phi_CNOT = DensityMatrix(n5)

drawOperator(U_phi_CNOT)

<IPython.core.display.Latex object>

#### Calculate U<sub>ϕ</sub> for Control X^(1/2)

In [31]:
v0 = exponential3(np.pi/8, I8)
v1 = exponential3(np.pi/8, IZZ)
v2 = exponential3(np.pi/8, ZIZ)
v3 = exponential3(np.pi/8, ZZI)
v4 = exponential3(-np.pi/8, IIZ)
v5 = exponential3(-np.pi/8, IZI)
v6 = exponential3(-np.pi/8, ZII)

In [32]:
n0 = mul(v1,v0)
n1 = mul(v2,n0)
n2 = mul(v3,n1)
n3 = mul(v4,n2)
n4 = mul(v5,n3)
n5 = mul(v6,n4)

U_phi_SX = DensityMatrix(n5)

drawOperator(U_phi_SX)

<IPython.core.display.Latex object>

#### Calculate U_CNOT_12

In [28]:
b1 = exponential3(np.pi/4, IYI)
b2 = exponential3(-np.pi/4, IIZ)
b3 = exponential3(np.pi/4, S2S3)
b4 = exponential3(np.pi/4, S1S3)
b5 = U_phi_CNOT
b6 = exponential3(-np.pi/4, IYI)

In [29]:
o1 = mul(b2,b1)
o2 = mul(b3,o1)
o3 = mul(b4,o2)
o4 = mul(b5,o3)
o5 = mul(b6,o4)



U_CNOT_12 = DensityMatrix(o5)

drawOperator(U_CNOT_12)

<IPython.core.display.Latex object>

#### Calculate U_SX_23

In [33]:
b1 = exponential3(np.pi/4, IIY)
b2 = exponential3(np.pi/8, ZII)
b3 = exponential3(-np.pi/8, S1S3)
b4 = exponential3(-np.pi/8, S1S2)
b5 = U_phi_SX
b6 = exponential3(-np.pi/4, IIY)

In [34]:
o1 = mul(b2,b1)
o2 = mul(b3,o1)
o3 = mul(b4,o2)
o4 = mul(b5,o3)
o5 = mul(b6,o4)


U_SX_23 = DensityMatrix(o5)

drawOperator(U_SX_23)

<IPython.core.display.Latex object>

#### Calculate U_SX_23_dagger

In [35]:
U_SX_23_dagger = U_SX_23.conjugate()
drawOperator(DensityMatrix(U_SX_23_dagger))

<IPython.core.display.Latex object>

#### Calculate U_SX_13

In [36]:
b1 = exponential3(np.pi/4, IIY)
b2 = exponential3(np.pi/8, IZI)
b3 = exponential3(-np.pi/8, S2S3)
b4 = exponential3(-np.pi/8, S1S2)
b5 = U_phi_SX
b6 = exponential3(-np.pi/4, IIY)

In [37]:
o1 = mul(b2,b1)
o2 = mul(b3,o1)
o3 = mul(b4,o2)
o4 = mul(b5,o3)
o5 = mul(b6,o4)

U_SX_13 = DensityMatrix(o5)

drawOperator(U_SX_13)

<IPython.core.display.Latex object>

## Construction of final CCNOT

In [40]:
q1 = mul(U_CNOT_12, U_SX_23)
q2 = mul(U_SX_23_dagger, q1)
q3 = mul(U_CNOT_12, q2)
q4 = mul(U_SX_13, q3)

U_CCNOT = DensityMatrix(q4)

In [43]:
drawOperator(U_CCNOT)

<IPython.core.display.Latex object>